In [1]:
class Entidad:
    """
    Clase que modela las entidades que desean  comunicarse.


    Attributes
    ----------
    nombre: string
        Nombre identificatorio de la entidad.
   
    Methods
    -------
    
    """
    def __init__(self,nombre=''):
        self.nombre = nombre
        self.dh_parameters = {
            "x": 0,
            "base": 0
        }
        
    def generate_dh_parameters(self,dh_parameters,number_bytes=3):
        self.dh_parameters["x"] = secrets.randbits(number_bytes)
        self.dh_parameters["base"] = expModular(
            self.dh_parameters["x"],
            dh_parameters["alpha"],
            dh_parameters["p"]
        )
        

In [2]:
class DH:
    """
    Clase que representa el algoritmo Deffie - Hellman.


    Attributes
    ----------
    
    Methods
    -------
    
    """
    
    def __init__(self,number_bytes=8):
        self.number_bytes = number_bytes
        self.parameters = {
            "p" : 0,
            "alpha" : 0
        }
        self.generate_parameters()
    
    def generate_parameters(self):
        self.parameters["p"] = obtenerPrimo(bits = self.number_bytes * 8)
        self.parameters["alpha"] = generador(p = self.parameters["p"])
        

In [3]:
import math

import secrets
## --> secrets
## https://docs.python.org/3/library/secrets.html
## The secrets module is used for generating cryptographically strong random numbers suitable for managing data
## such as passwords, account authentication, security tokens, and related secrets.

## secrets.randbelow(n)
##      Return a random int in the range [0, n).

## secrets.randbits(k)
##      Return an int with k random bits.



def obtenerPrimo(bits):
    """
    Genera aleatoriamente un número y verifica si es primo.
    De no serlo la función se llama de nuevo.
   
    Attributes
    ----------
    bits -- Cantidad de bits.
    
    Return:
    Int -- Número primo.
    
    """
    p = secrets.randbits(bits)
    if esPrimo(p)==True:
        return p
    else:
        return obtenerPrimo(bits)

def esPrimo(n):
    """Devuelve si el parámetro "n" es primo o no.
    
    Parámetros:
    n -- Número entero positivo.

    Return:
    Bool -- True si es primo, False si es compuesto.
    
    """
    if n<2:
        return False
    if n>2 and n%2==0:
        return False
    for i in range(3,round(math.sqrt(n))+1,2):
        if(n%i==0):
            return False
    return True

def generador(p):
    """Devuelve un generador del grupo p: g, talque 2 <= g <= p-2 y mcd(g,p) = 1

    Fuente: https://cp-algorithms.com/algebra/primitive-root.html

    First, find ϕ(n) and factorize it.
    Then iterate through all numbers g∈[1,n], and for each number, to check if it is primitive root, we do the following:

    Calculate all g^(ϕ(n)pi)( mod n ).
    If all the calculated values are different from 1, then g is a primitive root.

    Parámetros:
    p -- Número entero positivo primo.

    Return:
    Int -- Generador del grupo
    
    """
    q = [row[0] for row in factores(p-1)]
    for i in range(2,p):
        if mcd(p,i)==1:
            corte = 1
            for j in range(0,len(q)):
                aux = expModular(i,(p-1)/q[j],p)
                if aux==1:
                    corte = 0
                    break
            if corte == 1:
                return i

def factores(n):
    """Devuelve un array con los factores de un número entero.

    n= p_1^(e_1 ) * p_2^(e_2 ) * … *p_k^(e_k ) =>  [
														[p_1,e_1],
														[p_2,e_2],
														....
														[p_k,e_k]
													]

    Parámetros:
    n -- Número entero positivo.

    Return:
    Array[Int] -- Array de factores con exponentes de un número entero positivo.
    
    """
    lista = []
    if esPrimo(n):
        lista.append([n,1])
        return lista
    c = 0
    if n%2==0:
        while n%2 == 0:
            n = n // 2
            c = c + 1
        lista.append([2,c])
    i = 3
    while i<=n:
        c = 0
        if n%i==0:
            while n%i == 0:
                n = n // i
                c = c + 1
            lista.append([i,c])
        i = i + 2
    return lista

def mcd(a,b):
    """Devuelve el máximo comun divisor de 2 números.

    Parámetros:
    a -- Número entero positivo.
    b -- Número entero positivo.
    
    Return:
    Int -- Máximo comun divisor.
    
    """
    if b == 0:
        return a
    else:
        return mcd(b, a % b)
    
def expModular(a,b,n):
    """
    Sea n∈Z^+,n>1 y  a,b,x ϵ Z se requiere calcular el valor de x ≡ a^b  (mod n).

    Convertir el número b en su expresión binaria b = b_k b_(k-1)….b_2 b_1 b_0.
    Primero se analiza b_0, se asigna p = a  y:
        - Si b_0 = 1 almacenar c_0 ≡ p (mod n)
        - Si b_0 = 0 almacenar c_0 = 1.
    Luego ∀ i tal que 0<i≤k, se asigna p=p^2 y:
        - Si b_i = 1 almacenar c_i ≡ p (mod n).
        - Si b_i = 0 almacenar c_i = 1 .
    Se resolverá el valor x multiplicando entre si los c_i.


    Parámetros:
    a -- Número entero positivo.
    b -- Número entero positivo.
    n -- Número entero positivo, modulo.
    
    Return:
    Int -- Devuelve x.
     
    """
    c = 1
    p = a%n
    while b>0:
        if b%2==1:
            c = (c*p)%n
        b = b//2
        p = pow(p,2)%n
    return c

In [4]:
##defino las dos entidades que intercambiaran mensajes
A = Entidad("A") 
B = Entidad("B")

In [5]:
## genero el objeto diffie hellman, con sus parametros necesarios para la comunicación

dh = DH(number_bytes=3) 

dh.parameters

{'p': 12274489, 'alpha': 7}

In [6]:
# 𝐴 elige un 𝑥 𝜖 𝑁 de forma aleatoria y computa 𝑋 ≡ 𝛼𝑥 (𝑚𝑜𝑑 𝑝), y envía 𝑋 a 𝐵.
A.generate_dh_parameters(dh_parameters=dh.parameters,number_bytes=3)

A.dh_parameters


{'x': 3, 'base': 2187}